In [1]:
import pandas as pd
import numpy as np

movies = pd.read_csv('/Users/adrua/Desktop/Fariy of Choice/ml-latest-small/movies.csv')
ratings = pd.read_csv('/Users/adrua/Desktop/Fariy of Choice/ml-latest-small/ratings.csv')
print(movies.shape)
print(ratings.shape)


(9742, 3)
(100836, 4)


In [2]:
#user-item table로 변환
ratings = ratings[['userId', 'movieId', 'rating']]
ratings_matrix = ratings.pivot_table('rating', index='userId', columns='movieId')
ratings_matrix.head(3)

                                     

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#title 칼럼을 얻기 위해 movies와 조인
ratings_movies = pd.merge(ratings, movies, on='movieId')

#columns='title'로 title 칼럼으로 피벗 수행
ratings_matrix = ratings_movies.pivot_table('rating', index='userId', columns='title')

#NaN(평점 X) 값 모두 0 변환
ratings_matrix = ratings_matrix.fillna(0)

ratings_matrix.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 아이템 간 유사도 산출; 
코사인 유사도 기반, 사이킷런의 cosine_similarity()를 사용해 측정

이 함수는 행을 기준으로 서로 다른 행을 비교해 유사도를 산출 

so, userId기준인 행 레벨 데이터이므로 지금 consine_similiarity를 적용하면 아이템이 아닌 사용자 간 유사도가 만들어짐

현재의 ratings_matirx가 행; 아이템, 열;사용자 가 되어야 하므로, 
행렬 위치 변경 ➡️ Pandas의 transpose()함수 사용 


In [4]:
ratings_matrix_T = ratings_matrix.transpose()
ratings_matrix_T.head(3)

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


ratings_matrix를 전치 행렬 기반으로 변환한 데이터 세트를 기반으로 영화의 코사인 유사도 구하기 + 직관적인 영화 유사도 값을 표현하기 위해 Cosine_similarity()로 반환된 넘파이 행렬에 영화명을 매핑해 Dataframedmfh 변환

In [5]:
#코사인 유사도 구하기
from sklearn.metrics.pairwise import cosine_similarity
item_sim = cosine_similarity(ratings_matrix_T, ratings_matrix_T)

#Cosine similarity로 반환된 넘파이 행렬을 영화명을 매핑해 Dataframe으로 변환
item_sim_df = pd.DataFrame(data=item_sim, 
                           index=ratings_matrix.columns, 
                           columns=ratings_matrix.columns)

print(item_sim_df.shape)
item_sim_df.head(3)


(9719, 9719)


title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),1.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.141653,0.0,...,0.0,0.342055,0.543305,0.707107,0.0,0.0,0.139431,0.327327,0.0,0.0
'Hellboy': The Seeds of Creation (2004),0.0,1.000000,0.707107,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Round Midnight (1986),0.0,0.707107,1.000000,0.0,0.0,0.0,0.176777,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0


Godfather, The (1972), Inception (2010)과 유사도가 높은 상위 5개 영화 추출

In [6]:
item_sim_df["Godfather, The (1972)"].sort_values(ascending=False)[1:6]

title
Godfather: Part II, The (1974)               0.821773
Goodfellas (1990)                            0.664841
One Flew Over the Cuckoo's Nest (1975)       0.620536
Star Wars: Episode IV - A New Hope (1977)    0.595317
Fargo (1996)                                 0.588614
Name: Godfather, The (1972), dtype: float64

In [7]:
item_sim_df["Inception (2010)"].sort_values(ascending=False)[1:6]

title
Dark Knight, The (2008)          0.727263
Inglourious Basterds (2009)      0.646103
Shutter Island (2010)            0.617736
Dark Knight Rises, The (2012)    0.617504
Fight Club (1999)                0.615417
Name: Inception (2010), dtype: float64

### 아이템 유사도 데이터로 개인화된 영화 추천 알고리즘 만들기
위 유사도 데이터는 사용자 평점 기준이지만 영화 간 유사도만 가지고 영화를 추천한다. 
\+ 최근접 이웃 협업 필터링으로 개인에게 최적화된 영화 추천을 구현

➡️ 아직 관람하지 않은 영화에 대해서 아이템 유사도와 기존에 관람한 영화의 평점 데이터를 기반으로해 모든 영화의 예측 평점을 계산한 후 높은 예측 평점을 가진 영화를 추천

(설명추가 - p.620)
이때 N은 최근접 이웃 범위 계수(item neighbor)를 의미 ➡️ 특정 아이템과 유사도가 가장 높은 Top-N개의 다른 아이템을 추출하는 데 사용; 
먼저 N의 범위에 제약을 두지 않고 모든 아이템으로 가정하고 예측 평점을 구하는 로직을 작성한 뒤에, Top-N아이템을 기반으로 협업 필터링을 수행하는 로직으로 변경

---
영화 간 유사도를 가지는 Dataframe인 item_sim_df와 사용자-영화 평점 dataframe인 ratings_matrix변수를 활용해 사용자별로 최적화된 평점 스코어를 예측하는 함수 생성 
➡️ predict_rating()이며, 인자로 사용자-영화 평점 넘파이 행렬(Rating_matrix를 넘파이 행렬로 변환)과 영화 간 유사도를 가지는 넘파이 행렬(item_sim_df를 넘파이 행렬로 변환)을 입력받고 앞서 R^u,i 식으로 개인화된 예측 평점을 계산. 

---


In [8]:
#N의 범위에 제약을 두지 않는다면, 사용자별 영화 예측 평점 R_u,i는 사용자 u의 모든 영화에 대한 실제 평점과 
    #영화 i의 다른 모든 영화와의 코사인 유사도를 벡터 내적 곱(dot)한 값을 정규화를 위해 ***(p.620)로 나눈 것을 의미

def predict_rating(ratings_arr, item_sim_arr ):
    ratings_pred = ratings_arr.dot(item_sim_arr)/ np.array([np.abs(item_sim_arr).sum(axis=1)])
    return ratings_pred

In [9]:
ratings_pred = predict_rating(ratings_matrix.values, item_sim_df.values)
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index=ratings_matrix.index, \
                                  columns = ratings_matrix.columns)
ratings_pred_matrix.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.070345,0.577855,0.321696,0.227055,0.206958,0.194615,0.249883,0.102542,0.157084,0.178197,...,0.113608,0.181738,0.133962,0.128574,0.006179,0.212070,0.192921,0.136024,0.292955,0.720347
2,0.018260,0.042744,0.018861,0.000000,0.000000,0.035995,0.013413,0.002314,0.032213,0.014863,...,0.015640,0.020855,0.020119,0.015745,0.049983,0.014876,0.021616,0.024528,0.017563,0.000000
3,0.011884,0.030279,0.064437,0.003762,0.003749,0.002722,0.014625,0.002085,0.005666,0.006272,...,0.006923,0.011665,0.011800,0.012225,0.000000,0.008194,0.007017,0.009229,0.010420,0.084501


예측 평점이 실제 평점과 영화의 코사인 유사도를 내적(dot)한 값이기 때문에, 기존에 영화를 관람하지 않아 0에 해당했던 실제 영화 평점이 예측에서는 값이 부여되는 경우가 많이 발생한다. (예측 평점이 실 평점에 비해 작을 수 있음) 이는 내적 결과를 코사인 유사도 벡터 합으로 나누었기 때문에 생기는 현상

➡️ 예측 결과가 실제 평점과 얼마나 차이가 있는지 확인 
: 예측 평가 지표는 MSE(평균제곱오차)를 적용 

이때 유의점은, 사요자가 영화 평점을 주지 않은 경우 앞에서는 평점을 0으로 부과했지만, 앞에서 개인화된 예측 점수는 평점을 주지 않은 영화에 대해서도 아이템 유사도에 기반해 평점을 예측함. 따라서 여기서 실제와 예측 평점의 차이는 기존에 평점에 부여된 데이터에 대해서만 오차 정도를 측정

In [10]:
from sklearn.metrics import mean_squared_error

#사용자가 평점을 부여한 영화에 대해서만 예측 성능 평가 MSE를 구함 
def get_mse(pred, actual):
    #평점이 있는 영화만 추출
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

print('아이템 기반 모든 최근접 이웃 MSE: ', get_mse(ratings_pred, ratings_matrix.values))

아이템 기반 모든 최근접 이웃 MSE:  9.895354759094706


MSE = 9.9: 실제값과 예측값은 서로 스케일이 다르기 때문에 MSE가 클 수도 있다. 중요한 것은 MSE를 감소시키는 방향으로 개선하는 것

➡️ 앞의 predict_rating( ) 함수는 사용자별 영화의 예측 평점을 계산하기 위해 해당 영화와 다른 모든 영화 간의 유사도 벡터를 적용한 것.
많은 영화의 유사도 벡터를 이용하다 보니 상대적으로 평점 예측이 떨어짐. 

➡️ 특정 영화와 가장 비슷한 유사도를 가지는 영화에 대해서만 유사도 벡터를 적용하는 함수로 변경 
: predict_rating_topsim(ratings_arr, item_sim_arr, N=20)함수는 N인자를 가지고 있어 TOP-N 유사도를 가지는 영화 유사도 벡터만 예측값을 계산하는 데 적용

*다만 개별 예측값을 구하기 위해 행렬별로 for loop 반복 수행; TOP-N 유사도 벡터를 계산해야 하기 때문에 수행시간이 오래 걸림.


In [11]:
def predict_rating_topsim(ratings_arr, item_sim_arr, n=20):
    #사용자-아이템 평점 행렬 크기만큼 0으로 채운 예측 행렬 초기화
    
    pred = np.zeros(ratings_arr.shape)
    
    #열 크기만큼 루프 수행
    for col in range(ratings_arr.shape[1]):
        
        #유사도 행렬에서 유사도가 큰 순으로 n개 데이터 행렬의 인덱스 반환
        top_n_items = [np.argsort(item_sim_arr[:, col])[:-n-1:-1]]
        
        #개인화된 예측 평점을 계산
        for row in range(ratings_arr.shape[0]):
            pred[row, col] = item_sim_arr[col, :][top_n_items].dot(ratings_arr[row,:][top_n_items].T)
            pred[row, col] /= np.sum(np.abs(item_sim_arr[col,:][top_n_items]))
    
    return pred

In [12]:
#predict_rating_topsim()함수로 예측 평점 계산, 실제 평점과의 MSE를 구하기 & 예측 평점 넘파이 행렬은 판다스 Dataframe으로 재생성
ratings_pred = predict_rating_topsim(ratings_matrix.values, item_sim_df.values, n=20)
print('아이템 기반 최근접 TOP-20 이웃 MSE: ', get_mse(ratings_pred, ratings_matrix.values))

#예측 평점 데이터는 Dataframe으로 재생성 

ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index=ratings_matrix.index, columns=ratings_matrix.columns)

/var/folders/sb/ggzbw3nx66b6nbvcr0yybr2w0000gn/T/ipykernel_21946/1830049633.py:14: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  pred[row, col] = item_sim_arr[col, :][top_n_items].dot(ratings_arr[row,:][top_n_items].T)
/var/folders/sb/ggzbw3nx66b6nbvcr0yybr2w0000gn/T/ipykernel_21946/1830049633.py:15: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  pred[row, col] /= np.sum(np.abs(item_sim_arr[col,:][top_n_items]))


아이템 기반 최근접 TOP-20 이웃 MSE:  3.6949696289030554


➡️ 기존 9.89보다 향상됨(3.69)

### 특정 사용자에 대한 영화 추천; 
userId = 9인 사용자가 어떤 영화에 평점을 높이 주었는지 확인

In [13]:
user_rating_id = ratings_matrix.loc[9, :]
user_rating_id[ user_rating_id > 0].sort_values(ascending=False)[:10]

title
Adaptation (2002)                                                                 5.0
Citizen Kane (1941)                                                               5.0
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)    5.0
Producers, The (1968)                                                             5.0
Lord of the Rings: The Two Towers, The (2002)                                     5.0
Lord of the Rings: The Fellowship of the Ring, The (2001)                         5.0
Back to the Future (1985)                                                         5.0
Austin Powers in Goldmember (2002)                                                5.0
Minority Report (2002)                                                            4.0
Witness (1985)                                                                    4.0
Name: 9, dtype: float64

이 사용자에게 아이템 기반 협업 필터링을 통해 영화를 추천 - 이미 평점을 준 영화를 제외하고 추천할 수 있도록, 평점을 주지 않은 영화를 리스트 객체로 반환하는 함수인 get_unseen_movies( )를 정의

In [14]:
def get_unseen_movies(ratings_matrix, userId):
    #userId로 입력받은 사용자의 모든 영화 정보를 추출해 Series로 반환
    #반환된 user_rating은 영화명(title)을 인덱스로 가지는 Series 객체
    user_rating = ratings_matrix.loc[userId, :]
    
    #user_rating이 0보다 크면 기존에 관람한 영화이다. 대상 인덱스를 추출해 list 객체로 만든다
    already_seen = user_rating[user_rating > 0].index.tolist()
    
    #모든 영화명을 list객체로 만듦
    movies_list = ratings_matrix.columns.tolist()
    
    #list comprehension으로 already_seen에 해당하는 영화는 movies_list에서 제외함
    unseen_list = [movie for movie in movies_list if movie not in already_seen]
    
    return unseen_list

사용자가 영화의 평점을 주지 않은 추천 대상 영화 정보와 predict_rating_topsim()에서 추출한 사용자별 아이템 유사도에 기반한 예측 평점 데이터셋을 이용해 최종적으로 사용자에게 영화를 추천하는 함수인 recomm_movie_by_userid( )를 정의

해당 함수는 예측 평점 DataFrame과 추천하려는 사용자 id, 추천 후보 영화 리스트, 추천 상위 영화 개수를 인자로 받아서 사용자가 좋아할만한 가장 높은 예측 평점을 가진 영화를 추천

In [23]:
def recomm_movie_by_userid(pred_df, userId, unseen_list, top_n=10):
    #예측 평점 Dataframe에서 사용자id 인덱스와 unseen_list로 들어온 영화명 칼럼을 추출
    #가장 예측 평점이 높은 순으로 정렬
    recomm_movies = pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]
    return recomm_movies
    
#사용자가 관람하지 않는 영화명 추출
unseen_list = get_unseen_movies(ratings_matrix, 9)


#아이템 기반 최근접 이웃 협업 필터링으로 생성
recomm_movies = recomm_movie_by_userid(ratings_pred_matrix, 9, unseen_list, top_n=10)

recomm_movies = pd.DataFrame(data = recomm_movies.values, index = recomm_movies.index, columns=['pred_score'])

recomm_movies

,pred_score
title,
Shrek (2001),0.866202
Spider-Man (2002),0.857854
"Last Samurai, The (2003)",0.817473
Indiana Jones and the Temple of Doom (1984),0.816626
"Matrix Reloaded, The (2003)",0.800990
Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001),0.765159
Gladiator (2000),0.740956
"Matrix, The (1999)",0.732693
Pirates of the Caribbean: The Curse of the Black Pearl (2003),0.689591
